# skills in each job

In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os

import warnings
warnings.filterwarnings('ignore')

import time
import timeit

#import ENG
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

import pythainlp
from pythainlp import word_tokenize

import re

from __future__ import unicode_literals


def splitword(text):
  out=[]
  couUp=0
  left=0
  initial=text.strip()
  upper=re.findall('[A-Z]',initial)
  for i in range(len(initial)):
    if initial[i] in upper:
      couUp+=1
    if couUp==len(upper):
      out.append(initial[left:i])
      out.append(initial[i:])
      break
    elif initial[i] in upper and i !=0:
      out.append(initial[left:i])
      left=i
  return ' '.join(out).strip()

#Data Pre-processing for English only
# input = many words
#output = [w1, w2, w3, ......]

#1. Tokenization: Split the text into sentences and the sentences into words.
# Lowercase the words and remove punctuation.
#2. Words that have fewer than 3 characters are removed.
#3. All stopwords are removed.
#4. Words are lemmatized — 
#words in third person are changed to first person and verbs in past and future tenses are changed into present.
#5. Words are stemmed — words are reduced to their root form.
def lemmatize_stemming(text):
    return SnowballStemmer('english').stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

#process for all data
#input = docs
# output = a list of each word
def preprocessAll(text):
  text=str(text)
  text=text.strip()
  thai=pythainlp.thai_characters
  th=''
  en=''
  for i in range(len(text)):
    if text[i] in thai:
      th=th+text[i]
    else:
      if (i != 0) and (text[i-1] in thai):
        en=en+' '+text[i]
      else:
        en=en+text[i]
  temp_en=[]
  for i in en.split(' '):
    if i !='':
      temp_en.append(splitword(i))
  temp_en=' '.join(temp_en)
  return preprocess(temp_en)+word_tokenize(th, keep_whitespace=False)




[nltk_data] Downloading package wordnet to /home/std/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#model_tag, description
out=dict()
path='/home/std/Downloads/python/test_result/piplineV2_overfit/'
for file in os.listdir(path):
    start = time.time()
    data=pd.read_csv(path+file)
    for i in range(len(data)):
        if data.iloc[i]['model_tag'] not in out.keys():
            out[data.iloc[i]['model_tag']] = dict()
            for word in preprocessAll(data.iloc[i]['description']):
                if word not in out[data.iloc[i]['model_tag']].keys():
                    out[data.iloc[i]['model_tag']][word]=1
                else:
                    out[data.iloc[i]['model_tag']][word]+=1
        else:
            for word in preprocessAll(data.iloc[i]['description']):
                if word not in out[data.iloc[i]['model_tag']].keys():
                    out[data.iloc[i]['model_tag']][word]=1
                else:
                    out[data.iloc[i]['model_tag']][word]+=1
    stop=time.time()
    print('time(sec): ',stop - start)

time(sec):  1484.3183846473694
time(sec):  1149.2855005264282
time(sec):  1329.4879112243652
time(sec):  2004.197987794876
time(sec):  2041.4818713665009
time(sec):  1713.7281403541565
time(sec):  1177.33740067482
time(sec):  1044.3301358222961


In [ ]:
data['description'].iloc[2]

'ฝึกงานด้านงาน- IT Development- Marketing- CRM (Customer Relation Management)- VRM (Vendor Relationship Management)- Office Environment and Assets- Account Executive (Sales Coordinator)- HR'

In [ ]:
#out.keys() = all jobs
#out[job] = dict(word: count)
jobs=list(out.keys())
jobs

['coordinator',
 'Investor // finance // account',
 'Others',
 'marketing // advertising',
 'Policy manager // BD',
 'Stock // procurement // supply',
 'teacher',
 'law',
 'cooker + bartender',
 'translator',
 'HR',
 'sale // service',
 'manufacturing manager',
 'writer',
 'security + safety',
 'credit analyst',
 'graphic design',
 'retail manager',
 'maid',
 'therapist',
 'waiter',
 'architecture',
 'vehicle driver',
 'care taker',
 'cashier']

In [ ]:
len(jobs)

25

In [ ]:
delete_words=['และ', 'การ', 'ที่','ได้', 'การ', 'งาน', 'กับ', 'ต่างๆ', 'ของ', 'ใน', 'ให้','อื่นๆ', 'จัดทำ',
              'เกี่ยวข้อง', 'ตามที่','บริษัท','ด้าน','มี','เพื่อ','รับ','สามารถ','หรือ',
             'ทำ','จาก','ทั้ง','ฝ่าย', 'ส่ง','ระบบ','จัด','ตาม','แผนก', 'ใช้','เป็น','วัน','ดี','อย่าง',
                 'เรื่อง','ทาง','ปี','เช่น','ผู้บังคับบัญชา','การทำงาน','ทั่วไป','ต่อ','บริษัทฯ','สาขา','เวลา',
                 'หน้าที่','เกี่ยวับ','จันทร์','น','ระหว่าง','เบิก','ไม่','จะ','skill', 'good','experi',
                 'year','task','requir','depart','ensur','organ','degre','qualif','person','time',
                 'intern','field','includ','duti','bangkok','staff','perform','abl','applic','salari',
                 'abil','general','follow','posit','candid','thailand','updat','command','order',
                 'ควบคุม','ประจำเดือน','รายการ','ออก','รายละเอียด','ราย','หัก','ทำงาน','ปฏิบัติงาน',
                 'รับผิดชอบ','ณ','ใบสำคัญ','ปิด','กำหนด','นำ','prepar','work','month','control',
                 'relat','process','provid', 'close','knowledg','project','strong','assign','thai',
                 'function','activ','maintain','procedur','standard','improv','annual','extern',
                 'issu','submit','contact','prefer','flow','receiv','appli','รับมอบหมาย','พนักงาน',
                 'หน่วยงาน','รายงาน','ขึ้นไป','เกี่ยวกับ','อายุ','ร้าน','ใหม่','คุณสมบัติ','โดย','ความ','อุปกรณ์',
                 'ค่า','แก่','พื้นที่','implement','food','program','level','drive','target','achiev',
                 'build','achiev','effect','channel','understand','oper','need','identifi','high',
                 'partner','role','offic','problem','resourc','direct','technic','success','chang',
                 'initi','expect','review','area','construct','environ','solv','peopl','goal',
                 'technolog','stakehold','object','creat','result','base','ถูกต้อง','ต่างประเทศ','ราคา',
                 'สรุป','chain','procur','sourc','locat','resum','contract','ระดับ','ผู้','สำหรับ','ได้รับ',
                 'ติดต่อ','ข้อมูล','head','compani','individu','member','มีประสบการณ์','firm','relev',
                 'case','board','road','group','practic','descript','unit','หัวหน้างาน','เตรียม',
                 'มาตรฐาน','ตำแหน่ง','คุณภาพ','บาท','หญิง','เรียบร้อย','เพศชาย','ชั่วโมง','ไทย','ม','อยู่',
                 'คุณ','ผู้ช่วย','มา','วุฒิ','ความรับผิดชอบ','สูตร','พร้อม','เดือน','รวมถึง','เป็นพิเศษ','ช่วย','ใจรัก',
                 'สัปดาห์','ทันที','เวลาทำงาน','พนักงานประจำ','ให้ได้','มีความรู้','เลือก','ศุกร์','ผู้สมัคร','ชอบ',
                 'ลักษณะ','เริ่ม','ภายใต้','มีประสิทธิภาพ','ปฏิบัติหน้าที่','accid','occup','environment','equip',
                 'qualiti','workplac','femal','client','nation','hour','custom','floor','sukhumvit',
                 'necessari','klongton','advantag','room','vice','เงินเดือน','ภายใน','ดำเนินการ','วางแผน',
                 'ประจำปี','องค์กร','สวัสดิการ','พัฒนา','conduct','benefit','engag','เสาร์','ติดตาม','ฐาน','หา',
                 'โทร','เป้าหมาย','โทรศัพท์','กลุ่ม','หยุด','ผ่าน','ประจำ','ทุก','ไป','สร้าง','เขต','วันหยุด',
                 'ชม','เงิน','เข้า','อาทิตย์','เป้า','เก่า','เพิ่ม','ผล','ประเภท','ทักษะ','เป็นไปตาม','site',
                 'specif','accord','effici','action','minimum','เนื้อหา','ติด','ช่องทาง','ติด','บน','เรา',
                 'ลง','ๆ','หาก','ได้ดี','จำกัด','รูปแบบ','กิจกรรม','จบ','น่าสนใจ','ความสามารถ','ต้อง','สนใจ',
                 'การศึกษา','ร่วม','บิ๊กซี','ด้วย','เพื่อให้','ทีมงาน','googl','platform','แผนงาน','จัดการ',
                 'โครงการ','รวมทั้ง','เกิด','แผน','อัน','เกิดเหตุ','สถาน','ภายนอก','ประสบ','ขั้นตอน','ลูกค้า',
                 'ประกอบ','อันต','ดำเนินงาน','บริการ','กิจการ','เนื่องจาก','ดับ','เสนอ','ปัญหา','ปฏิบัติ','เมื่อ',
                 'สาเหตุ','ดำเนิน','เดือดร้อน','รำคาญ','พิจารณา','เจ้าหน้าที่','ความพร้อม','เปิด','จ่าย',
                 'เป็นไป','นโยบาย','กำกับ','โรงงาน','เหตุ','เช็ค','มอบหมาย','คู่มือ','บุคคล','ปฎิบัติ','วุฒิการศึกษา',
                 'ชี้','ร่วมกับ','ที่อยู่','เหมาะสม','จุด','พบ','แก้ไข','สภาพแวดล้อม','ขยาย','สมัคร','ขอ','แต่ละ',
                 'ไว้','เกิน','เบื้องต้น','หลักเกณฑ์','ตรง','รักษา','ก่อน','น้อย','เงื่อนไข','ผลิต','คิด','ความต้องการ',
                 'deadlin','premier','อาคาร','สูงสุด','ขึ้น','สถานี','เพิ่มเติม','รัก','ยา','ชั้น','ชุด','หน้า',
                 'บริเวณ','ถูก','สำนักงาน','พื้น','เสมอ','ใช้งาน','ต้องตาม','ตั้งแต่','ทุกชนิด','การใช้งาน','เข้างาน',
                 'ผนัง','ฝ้า','เพดาน','โต๊ะ','รู้จัก','น้ำยา','ทัน','สถานที่','ปฏิบัติตาม','น้ำ','เก็บ','สภาพ','ผู้บริหาร',
                 'ส่วนกลาง','ละ','จำนวน','ขยะ','รอบ','ออฟฟิศ','วาง','ส่วน','มนุษย์ ','recent','detail','ท',
                 'อำเภอ','โลตัส','นท','รี','นครราชสีมา','แล้ว','ทราบ','เซ','อ','เม้นท์','รัล','จังหวัด',
                 'ชลบุรี','เทสโก้','รีต','รายได้','เลย','เบี้ย','ภาค','ลด','หลัง','ระยอง','กรณี','รอย',
                 'เป็นต้น','ที่มา','บุรีรัมย์','ทรู','หนอง','ตัว','ต่ำกว่า','โรบินสัน','คน','ฟรี','เม','เชียงใหม่',
                 'เข้ามา','เครื่องมือ','รับสมัคร','์','ขอนแก่น','สมบูรณ์','อุดรธานี','extra','fuction','true','ตัวจริง',
                 'โบนัส','บางนา','กองทุนสำรองเลี้ยงชีพ','แขวง','รางวัล','บัตรประชาชน','ริ','เส','ถึง','สำเนา',
                 'กรุงเทพมหานคร','รูปถ่าย','ถ','ครั้ง','ประกันสังคม','เอ็ม','เค','มหาชน','ฟอร์ม','เงินช่วยเหลือ',
                 'รูป','นิ้ว','กรุ๊ป','ตราด','การสมัครงาน','ขนาด','นาน','ได้ที่','จำเป็นต้อง','ฟ','กองทุน','ผู้อื่น','วันอาทิตย์',
                 'นอก','ชัดเจน','ถนน','ปรับ','ติดต่อสอบถาม','http','limit','button','mkrestaur','โครง','สอดคล้อง',
                 'ประสบการณ์','สูง','แล้วเสร็จ','ศึกษา','ประชุม','ความถูกต้อง','เคลียร์','ปรับปรุง','ขยัน','ทางใน','อาหาร',
                 'ปริมณฑล','รู้เส้น','กรุงเทพ','ต่างจังหวัด','กรุงเทพฯ','นาย','บ้าน','ยัง','บ','ห้าง','สุขภาพ','พร','นี้',
                 'ประ','ต์','ตนเอง','รอยสัก','ร่มผ้า','เงินโบนัส','เล่ม','สังคม','เวลาเปิด','ห้วยขวาง','ความสะอาด',
                 'วิ่ง','interest','public','incent','ให้การ','จัดเตรียม','ห้อง','สิทธิ','ศูนย์','อื่น','สังเกต','เวช',
                 'ทางการ','สั่งอาหาร','ขาด','แทน','แต่ละวัน','แยก','ประจำวัน','ทำการ','พ','ละเอียด','สำนักงานใหญ่',
                 'แจ้ง','รูด','บาร์','ทุกวัน','ตาราง','ป้ายราคา','เติม','คืน','แก้ไขปัญหา','งานประจำ','ฯ','ซ่อม',
                 'center','home','tender','nonthaburi','silom','salarieswith','bangkhen','muang','photograph']

In [ ]:


def top_words(data, job):
    print(job+'\n'*2+'top Thai words')
    out=data[job]
    #print(tot)
    sort_thai={k: v for k, v in sorted(out.items(), key=lambda item: item[1], reverse=True)}
    sort_eng={k: v for k, v in sorted(out.items(), key=lambda item: item[1], reverse=True)}
    tot=sum(list(sort_thai.values()))
    #g=pd.DataFrame(list(sort.items()),columns = ['word','count'])
    #print(g[:50])
    thai=[]
    eng=[]
    for word in sort_thai.keys():
        if word[0] not in 'abcdefghijklmnopqrstuvwxyz':
            thai.append(word)
        else:
            eng.append(word)
    for i in eng:
        if i in delete_words:
            del sort_eng[i]
        del sort_thai[i]
    for j in thai:
        if j in delete_words:
            del sort_thai[j]
        del sort_eng[j]
    tha=pd.DataFrame(list(sort_thai.items()),columns = ['word','percent'])
    tha['percent']=(tha['percent']/tot)*100
    tha=tha[tha['percent']>=0.1]
    if len(tha) <=50:
        print(tha)
    else:
        print(tha[:50])
    en=pd.DataFrame(list(sort_eng.items()),columns = ['word','percent'])
    en['percent']=(en['percent']/tot)*100
    en=en[en['percent']>=0.1]
    print('\n')
    print('top English words plot')
    if len(en)<=50:
        print(en)
    else:
        print(en[:50])
    #plt.figure(figsize=(15,60))
    #sns.barplot(x='percent',y='word',data=en)
    print('#'*80)
    print('\n'+'\n')

In [ ]:
for job in jobs:
    top_words(out, job)

coordinator

top Thai words
         word   percent
0      เอกสาร  1.392409
1   ประสานงาน  1.205148
2        ดูแล  0.594713
3      บันทึก  0.225005
4     จัดเก็บ  0.224830
5     ตรวจสอบ  0.223225
6   งานเอกสาร  0.191723
7        คีย์  0.186181
8   การประชุม  0.181485
9         ขาย  0.164247
10     สินค้า  0.141583
11    นัดหมาย  0.120699
12     รวบรวม  0.107719


top English words plot
          word   percent
0        manag  0.730842
1    administr  0.498283
2      support  0.492157
3     document  0.433762
4      coordin  0.393073
5         meet  0.390797
6       assist  0.334648
7       report  0.328844
8      english  0.323010
9         team  0.308543
10    communic  0.286812
11      servic  0.266686
12       excel  0.256565
13        busi  0.238860
14       handl  0.233434
15      arrang  0.218646
16        sale  0.212229
17      inform  0.202662
18    bachelor  0.197353
19     respons  0.179035
20      execut  0.178044
21   secretari  0.176614
22       write  0.167660
23      tra

# jobs similarity

In [ ]:
########################## similarity
def CouIntersec(a,b):
    temp_a = a.copy()
    temp_b = b.copy()
    cou=0
    for i in temp_a:
        if i in temp_b:
            cou+=1
            index=temp_b.index(i)
            temp_b.pop(index)
    return cou

In [ ]:
a=[1,2,3,4,5,6]
b=[1,1,1,5,6]
CouIntersec(a,b)

3

In [ ]:
def all_words(data, job):
    out=data[job]
    sort={k: v for k, v in sorted(out.items(), key=lambda item: item[1], reverse=True)}
    tot=sum(list(sort.values()))
    g=pd.DataFrame(list(sort.items()),columns = ['word','percent'])
    g['percent']=(g['percent']/tot)*100
    g=g[g['percent']>=0.1]
    
    tar=[]
    for word in list(g['word']):
        if word not in delete_words:
            tar.append(word)
            
    return tar


In [ ]:
all_words(out, jobs[0])

['เอกสาร',
 'ประสานงาน',
 'manag',
 'ดูแล',
 'administr',
 'support',
 'document',
 'coordin',
 'meet',
 'assist',
 'report',
 'english',
 'team',
 'communic',
 'servic',
 'excel',
 'busi',
 'handl',
 'บันทึก',
 'จัดเก็บ',
 'ตรวจสอบ',
 'arrang',
 'sale',
 'inform',
 'bachelor',
 'งานเอกสาร',
 'คีย์',
 'การประชุม',
 'respons',
 'execut',
 'secretari',
 'write',
 'ขาย',
 'travel',
 'file',
 'schedul',
 'สินค้า',
 'present',
 'word',
 'นัดหมาย',
 'record',
 'event',
 'request',
 'microsoft',
 'admin',
 'correspond',
 'point',
 'รวบรวม',
 'mail',
 'account',
 'speak',
 'appoint',
 'director']

In [ ]:
result=[]
key=list(out.keys())
for i in range(len(key)-1):
    for j in range(i+1, len(key)):
        a=all_words(out, jobs[i])
        b=all_words(out, jobs[j])
        result.append((CouIntersec(a,b),(jobs[i],jobs[j])))
result2=sorted(result, reverse=True)
len(result2)

300

In [ ]:
result2[:100]

[(45, ('marketing // advertising', 'Policy manager // BD')),
 (40, ('Policy manager // BD', 'manufacturing manager')),
 (38, ('Investor // finance // account', 'manufacturing manager')),
 (38, ('HR', 'manufacturing manager')),
 (36, ('writer', 'graphic design')),
 (36, ('marketing // advertising', 'manufacturing manager')),
 (36, ('marketing // advertising', 'HR')),
 (36, ('Stock // procurement // supply', 'manufacturing manager')),
 (36, ('Investor // finance // account', 'Policy manager // BD')),
 (36, ('Investor // finance // account', 'Others')),
 (35, ('Others', 'manufacturing manager')),
 (35, ('Investor // finance // account', 'marketing // advertising')),
 (35, ('Investor // finance // account', 'Stock // procurement // supply')),
 (34, ('marketing // advertising', 'Stock // procurement // supply')),
 (34, ('Policy manager // BD', 'HR')),
 (33, ('marketing // advertising', 'translator')),
 (33, ('law', 'HR')),
 (33, ('Others', 'Stock // procurement // supply')),
 (33, ('Investo